# **KG-LLM**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers==4.26.1
!pip install sentencepiece
!pip install transformers --upgrade
!pip install sentencepiece --force-reinstall
!pip install -U scikit-learn
!pip install --upgrade accelerate

# *KG-LLM(CoT)*

In [ ]:
import pandas as pd
import json
from collections import deque
import csv
import random
from sklearn.model_selection import train_test_split
import os


In [ ]:

input_file = open("train2id.txt", "r")
output_file = open("output.txt", "w")

# total number of lines
number = int(input_file.readline())

nodes = set()

graph = {}

for i in range(number):
    content = input_file.readline()
    node1, node2, relation = content.strip().split()

    nodes.add(node1)
    # nodes.add(node2)

    relation = int(relation)

    # Check if the first node already exists in the dictionary
    if node1 not in graph:
        # If not, create a new dictionary for the node
        graph[node1] = {}
    # Add the neighboring node and the relationship to the dictionary for node1
    graph[node1][node2] = relation


node_list = list(nodes)
node_list2 = list(nodes)

In [ ]:

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

relation2id = {}

with open("relation2id.txt", "r") as file:
    relations = int(file.readline())
    for line in file:
        relation, relation_id = line.strip().split("\t")
        relation2id[int(relation_id)] = relation

unique_rows = set()

# change size if you want, but larger the size, smaller the data
size = 30
# how many positive and negative data set you want to train, the more dataset, the more time to train
total = 60000
fieldnames = ['input_text', 'output_text']

instruction = 'Answer the following yes/no question by reasoning step-by-step. '

with open("train_data.csv", mode="w", newline='') as tra:
  with open("positive_data.csv", mode="w", newline='') as pos:
    with open("negative_data.csv",  mode="w", newline='') as neg:

        # Create a CSV writer object and write the headers to the file
        writer_pos = csv.DictWriter(pos, fieldnames=fieldnames)
        writer_pos.writeheader()

        writer_neg = csv.DictWriter(neg, fieldnames=fieldnames)
        writer_neg.writeheader()

        writer_tra = csv.DictWriter(tra, fieldnames=fieldnames)
        writer_tra.writeheader()


        def dfs(graph, size):
            pos_count = 0
            neg_count = 0
            times = 0
            term = True

            while times < total:
                visited = set()
                kg = []
                graph_size = random.randint(2, size)
                first_node = random.choice(node_list)
                visited.add(first_node)
                last_node = ""
                previous_node = first_node
                stack = [first_node]
                input_text = ""
                output_text = ""
                while len(visited) < graph_size:
                    if previous_node not in graph or set(graph[previous_node].keys()).issubset(visited):
                        node = random.choice(node_list)
                        while node in visited:
                            node = random.choice(node_list)
                        input_text += 'node_{} not connected with node_{}. '.format(previous_node, node)
                        output_text += 'node_{} not connected with node_{} means there is no relationship between node_{} and node_{}. '.format(previous_node, node, previous_node, node)
                        visited.add(node)
                        previous_node = node
                    else:
                        node = random.choice(list(graph[previous_node].keys()))
                        while node in visited:
                            node = random.choice(list(graph[previous_node].keys()))
                        relation = graph[previous_node][node]
                        r = relation2id[relation]
                        input_text += 'node_{} has relation_{} with node_{}. '.format(previous_node, relation, node)
                        output_text += 'node_{} has relation_{} with node_{}, means node_{} {} node_{}. '.format(previous_node, relation, node, previous_node, r, node)
                        visited.add(node)
                        previous_node = node
                    if len(visited) == graph_size:
                        last_node = previous_node

                # input_text += 'Answer the following yes/no question by reasoning step-by-step. Is the first node connnected with the last node?'
                was = len(unique_rows)
                unique_rows.add(input_text)
                if len(unique_rows) > was:
                  # if pos_count < int(total/2):
                    if first_node in graph and last_node in graph[first_node] and term:
                      # if pos_count < int(total/2):
                        final_relation = relation2id[graph[first_node][last_node]]
                        output_text += 'So node {} {} node {}. The answer is yes.'.format(first_node, final_relation, last_node)
                        prompt = 'Is node {} connnected with node {}?'.format(first_node, last_node)
                        writer_pos.writerow({'input_text': input_text + instruction + prompt, 'output_text': output_text})
                        writer_tra.writerow({'input_text': input_text + instruction + prompt, 'output_text': output_text})
                        pos_count += 1
                        term = False
                        times += 1
                    elif last_node in graph and first_node in graph[last_node] and term:
                      # if pos_count < int(total/2):
                        final_relation = relation2id[graph[last_node][first_node]]
                        output_text += 'So node {} {} node {}. The answer is yes.'.format(last_node, final_relation, first_node)
                        prompt = 'Is node {} connected with node {}?'.format(last_node, first_node)
                        writer_pos.writerow({'input_text': input_text + instruction + prompt, 'output_text': output_text})
                        writer_tra.writerow({'input_text': input_text + instruction + prompt, 'output_text': output_text})
                        pos_count += 1
                        term = False
                        times += 1
                  # elif neg_count < int(total//2):
                    elif not term:
                      # if neg_count < int(total/2):
                        output_text += 'So there is no connection between node {} and node {}. The answer is no.'.format(first_node, last_node)
                        prompt = 'Is node {} connected with node {}?'.format(first_node, last_node)
                        writer_neg.writerow({'input_text': input_text + instruction + prompt, 'output_text': output_text})
                        writer_tra.writerow({'input_text': input_text + instruction + prompt, 'output_text': output_text})
                        neg_count += 1
                        term = True
                        times += 1
                else:
                  continue


            print(pos_count)
            print(neg_count)

        dfs(graph, size)

30000
30000


In [ ]:
df1 = pd.read_csv('train_data.csv')
df1.sample(5)

,input_text,output_text
6139,node_605 has relation_0 with node_25492. node_...,"node_605 has relation_0 with node_25492, means..."
41783,node_8439 has relation_0 with node_1873. node_...,"node_8439 has relation_0 with node_1873, means..."
42034,node_16736 has relation_0 with node_16737. Ans...,"node_16736 has relation_0 with node_16737, mea..."
33259,node_33420 has relation_1 with node_16068. nod...,"node_33420 has relation_1 with node_16068, mea..."
49836,node_29938 has relation_4 with node_24391. Ans...,"node_29938 has relation_4 with node_24391, mea..."


In [ ]:
import csv

# Open the CSV file for reading and writing
with open('positive_data.csv', 'r+') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)

    # Create a set to store unique rows
    unique_rows = set()

    # Iterate over each row in the CSV file
    for row in reader:
        # Convert the row to a tuple and add it to the set
        unique_rows.add(tuple(row))

    # Clear the contents of the file and write the header row
    csvfile.seek(0)
    csvfile.truncate()
    writer = csv.writer(csvfile)
    writer.writerow(['input_text', 'output_text'])

    num_unique_rows = len(unique_rows)
    total_rows = reader.line_num

    print("Total rows:", total_rows)
    print("Unique rows:", num_unique_rows)

    # Write the unique rows to the file
    for row in unique_rows:
        writer.writerow(row)

Total rows: 30001
Unique rows: 30001


In [ ]:
# Open the CSV file for reading and writing
with open('negative_data.csv', 'r+') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)

    # Create a set to store unique rows
    unique_rows = set()

    # Iterate over each row in the CSV file
    for row in reader:
        # Convert the row to a tuple and add it to the set
        unique_rows.add(tuple(row))

    # Clear the contents of the file and write the header row
    csvfile.seek(0)
    csvfile.truncate()
    writer = csv.writer(csvfile)
    writer.writerow(['input_text', 'output_text'])

    num_unique_rows = len(unique_rows)
    total_rows = reader.line_num

    print("Total rows:", total_rows)
    print("Unique rows:", num_unique_rows)

    # Write the unique rows to the file
    for row in unique_rows:
        writer.writerow(row)


Total rows: 30001
Unique rows: 30001


In [ ]:
# Load the CSV file into a pandas DataFrame
data = pd.read_csv('positive_data.csv')

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2)

# Save the test data to a new CSV file
test_data.to_csv('pos_test.csv', index=False)

# Delete the test data from the original CSV file
data.drop(test_data.index, inplace=True)
data.to_csv('positive_train.csv', index=False)

In [ ]:
# Load the CSV file into a pandas DataFrame
data = pd.read_csv('negative_data.csv')

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2)

# Save the test data to a new CSV file
test_data.to_csv('neg_test.csv', index=False)

# Delete the test data from the original CSV file
data.drop(test_data.index, inplace=True)
data.to_csv('negative_train.csv', index=False)

In [ ]:
# combind the training data
df1 = pd.read_csv('positive_train.csv')
df2 = pd.read_csv('negative_train.csv')
combined_df = pd.concat([df1, df2], axis=0)
combined_df.to_csv('train.csv', index=False)

In [ ]:
# combind the testing data
df1 = pd.read_csv('pos_test.csv')
df2 = pd.read_csv('neg_test.csv')
combined_df = pd.concat([df1, df2], axis=0)
combined_df.to_csv('test.csv', index=False)

Train

In [ ]:

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd

# Load the FLAN-T5 model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base', model_max_length=512)

# Define the dataset class
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        # Get the input and output sequences
        input_sequence = self.data.iloc[idx]['input_text']
        output_sequence = self.data.iloc[idx]['output_text']

        # Add task-specific prefix to input sequence
        # input_sequence = 'is first node connect with last node: ' + input_sequence

        # Encode the input and output sequences using the T5 tokenizer
        input_encoding = tokenizer(input_sequence, padding='max_length', max_length=512, truncation=True, return_tensors='pt')
        output_encoding = tokenizer(output_sequence, padding='max_length', max_length=512, truncation=True, return_tensors='pt')

        # Get the input IDs, attention mask, and label IDs from the encodings
        input_ids = input_encoding['input_ids'].squeeze()
        attention_mask = input_encoding['attention_mask'].squeeze()
        label_ids = output_encoding['input_ids'].squeeze()

        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'label_ids': label_ids}

    def __len__(self):
        return len(self.data)

# Define the data collator function
def data_collator(batch):
    input_ids = torch.stack([example['input_ids'] for example in batch])
    attention_mask = torch.stack([example['attention_mask'] for example in batch])
    label_ids = torch.stack([example['label_ids'] for example in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': label_ids}

# Load the data
preprocess_data = pd.read_csv('train.csv')

# Split the data into train and validation sets
train_data = preprocess_data.sample(frac=0.8, random_state=1)
val_data = preprocess_data.drop(train_data.index)

# Create the datasets
train_dataset = MyDataset(train_data)
val_dataset = MyDataset(val_data)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=1000,
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()


**Link Prediction without ICL**

In [ ]:
import pandas as pd
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

preds1 = []
expec1 = []
input1 = []

# instruction = 'Answer the following yes/no question by reasoning step-by-step. '
# # prompt = 'Is the first node connnected with the last node?'
# prompt = 'Is the last node connnected with the first node?'



model_path = '/content/results/checkpoint-3000'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

test_df = pd.read_csv('/content/test.csv')
test_df = test_df.sample(1000)

correct = 0

total = 0
count = 0
for index, row in test_df.iterrows():
    input_text = row['input_text']
    expected_output = row['output_text']
# input_text = "node_11324 has relation_1 with node_11323. node_11323 has relation_1 with node_26334. " + instruction + prompt
# expected_output = "node_11324 has relation_1 with node_11323, means node_11324 _derivationally_related_form node_11323. node_11323 has relation_1 with node_26334, means node_11323 _derivationally_related_form node_26334. So node 11324 _derivationally_related_form node 26334. The answer is yes."
    # print(input_text)
    # print(expected_output)

    # 对输入进行编码
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # 进行预测并解码输出
    generation_config = {
        'max_new_tokens': 1500
    }

    outputs = model.generate(input_ids, **generation_config)
    # outputs = model.generate(input_ids)
    # output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(outputs)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(count, output_text.split()[-1])
    # count+=1
    # print(output_text.strip()[-5:])
    # print(output_text)
    if len(output_text) > 0:
        output_text = output_text.split()[-1]
    else:
        output_text = 'no.'
    # print(output_text)
    # preds1.append(output_text)
    # expec1.append(expected_output.split()[-1])
    # input1.append(input_text.split())

    output_text_new = output_text.replace(".", "").replace("_", "")
    expected_output_new = expected_output.split()[-1].replace(".", "").replace("_", "")
    # print(output_text_new, expected_output_new)

    preds1.append(output_text_new)
    expec1.append(expected_output_new)
    input1.append(input_text.split())

    if expected_output_new == output_text_new:
        correct += 1
    total += 1

accuracy = correct / total


print(f"Accuracy: {accuracy}")

preds_test1 = np.array([1 if 'no' in label else 0 for label in preds1])
expec_test1 = np.array([1 if 'no' in label else 0 for label in expec1])
# preds_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in preds])
# expec_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in expec])
f1 = f1_score(expec_test1, preds_test1)

# print(preds[:50])
# print(expec[:50])

auc = roc_auc_score(expec_test1, preds_test1)

print(f"F1: {f1}")
print(f"AUC: {auc}")

Accuracy: 0.563
F1: 0.8458049886621316
AUC: 0.8631112391193037


An example of the KG-LLM(CoT) output with CoT reasoning process

In [ ]:
import pandas as pd
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

preds1 = []
expec1 = []
input1 = []

# instruction = 'Answer the following yes/no question by reasoning step-by-step. '
# # prompt = 'Is the first node connnected with the last node?'
# prompt = 'Is the last node connnected with the first node?'



model_path = '/content/results/checkpoint-3000'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

input_text = "node_16643 has relation_0 with node_13198. node_13198 has relation_1 with node_16256. Answer the following yes/no question by reasoning step-by-step. Is node 16643 connnected with node 16256?"
expected_output = "node_16643 has relation_0 with node_13198, means node_16643 _hypernym node_13198. node_13198 has relation_1 with node_16256, means node_13198 _derivationally_related_form node_16256. So node 16643 _derivationally_related_form node 16256. The answer is yes."
    # print(input_text)
    # print(expected_output)

# 对输入进行编码
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# 进行预测并解码输出
generation_config = {
    'max_new_tokens': 1500
}

outputs = model.generate(input_ids, **generation_config)

output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)
print(expected_output)
# "node_16643 has relation_0 with node_13198, means node_16643 _hypernym node_13198. node_13198 has relation_1 with node_16256, means node_13198 _derivationally_related_form node_16256. So node 16643 _derivationally_related_form node 16256. The answer is yes."

**Link Prediction with ICL**

In [ ]:
import csv
import pandas as pd

In [ ]:
df = pd.read_csv('test.csv')
count = len(df)

In [ ]:
c = 0

fieldnames = ['input_text', 'output_text']
with open("train.csv") as train:
  with open("context.csv", mode="w", newline='') as context:
      writer_context = csv.DictWriter(context, fieldnames=fieldnames)
      writer_context.writeheader()
      reader_train = csv.reader(train)
      # Iterate over each row in the CSV file
      for row in reader_train:
          input = row[0]
          output = row[1]
          if input == 'input_text' or output == 'output_text':
            continue

          input = input.split('.')
          input = input[:-2]
          input = '.'.join(input)

          output = output.split('.')
          words = output[-3:-1]
          # relationship = words[3]
          # print(words)
          words = '.'.join(words)

          if c < count and len(input) < 100:
            cont = 'Context: {}.'.format(input + '.' + words)
            writer_context.writerow({'input_text': cont, 'output_text': words})
            c += 1
            # print(c)



In [ ]:
# read csv files
df1 = pd.read_csv('context.csv')
df2 = pd.read_csv('test.csv')

# combine dataframes by interleaving rows

combined_df = pd.concat([df1, df2]).sort_index(kind='merge')

# write to new csv file
combined_df.to_csv('icl_link.csv', index=False)

In [ ]:
# df1 = pd.read_csv('context.csv')
df1 = pd.read_csv('icl_link.csv')

df1.sample(5)

,input_text,output_text
10890,node_63163 has relation_72 with node_2941. nod...,The answer is no.
948,node_55978 has relation_37 with node_24922. Is...,The answer is yes.
6210,node_5513 has relation_38 with node_1255. node...,The answer is no.
6675,node_5182 has relation_71 with node_2906. node...,The answer is no.
4980,node_43913 has relation_65 with node_2255. Is ...,The answer is yes.


In [ ]:
import pandas as pd
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

preds1 = []
expec1 = []
input1 = []

# instruction = 'Answer the following yes/no question by reasoning step-by-step. '
# # prompt = 'Is the first node connnected with the last node?'
# prompt = 'Is the last node connnected with the first node?'



model_path = '/common/home/tc822/Desktop/last/results/checkpoint-3000'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

test_df = pd.read_csv('/common/home/tc822/Desktop/last/icl_link.csv')

correct = 0

total = 0
count = 0
for index, row in test_df.iterrows():
    input_text = row['input_text']
    expected_output = row['output_text']
# input_text = "node_11324 has relation_1 with node_11323. node_11323 has relation_1 with node_26334. " + instruction + prompt
# expected_output = "node_11324 has relation_1 with node_11323, means node_11324 _derivationally_related_form node_11323. node_11323 has relation_1 with node_26334, means node_11323 _derivationally_related_form node_26334. So node 11324 _derivationally_related_form node 26334. The answer is yes."
    # print(input_text)
    # print(expected_output)

    # 对输入进行编码
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # 进行预测并解码输出
    generation_config = {
        'max_new_tokens': 1500
    }

    outputs = model.generate(input_ids, **generation_config)
    # outputs = model.generate(input_ids)
    # output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(outputs)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(count, output_text.split()[-1])
    # count+=1
    # print(output_text.strip()[-5:])
    # print(output_text)
    if len(output_text) > 0:
        output_text = output_text.split()[-1]
    else:
        output_text = 'no.'
    # print(output_text)
    # preds1.append(output_text)
    # expec1.append(expected_output.split()[-1])
    # input1.append(input_text.split())

    output_text_new = output_text.replace(".", "").replace("_", "")
    expected_output_new = expected_output.split()[-1].replace(".", "").replace("_", "")
    # print(output_text_new, expected_output_new)

    preds1.append(output_text_new)
    expec1.append(expected_output_new)
    input1.append(input_text.split())

    if expected_output_new == output_text_new:
        correct += 1
    total += 1

accuracy = correct / total


print(f"Accuracy: {accuracy}")

preds_test1 = np.array([1 if 'no' in label else 0 for label in preds1])
expec_test1 = np.array([1 if 'no' in label else 0 for label in expec1])
# preds_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in preds])
# expec_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in expec])
f1 = f1_score(expec_test1, preds_test1)

# print(preds[:50])
# print(expec[:50])

auc = roc_auc_score(expec_test1, preds_test1)

print(f"F1: {f1}")
print(f"AUC: {auc}")

# *Relation prediction without In-context learning*


In [ ]:
import csv
import pandas as pd
# context = 'Given node_2075 has relation_4 with node_12648. The relation between the first node and last node is _member_meronym. '
# instruction = 'Answer the following multiple-choice question by choosing one of these options: '
# option = ''
instruction = 'Answer the following question step-by-step. '
count = 0

question = 'The relationship between the first node and the last node is ?'
fieldnames = ['input_text', 'output_text']
with open("relation2id.txt", "r") as file:
    relations = int(file.readline())
    for line in file:
        relation, relation_id = line.strip().split("\t")
    #     instruction += relation + ', '
    # instruction = instruction[:-2] + '. '

with open("test.csv") as test:
    with open("relation.csv", mode="w", newline='') as icl:
      writer_icl = csv.DictWriter(icl, fieldnames=fieldnames)
      writer_icl.writeheader()
      reader = csv.reader(test)


      for row in reader:
          # print(row)
          input = row[0]
          output = row[1]
          if input == 'input_text' or output == 'output_text':
            continue
          input = input.split('.')
          input = input[:-2]
          input = '.'.join(input)
          # print(input)
          input = instruction + input + '. ' + question
          # input = input + '. ' + instruction + question

          output = output.split('.')
          words = output[-3].split()
          relationship = words[3]
          # print(relationship)
          output = output[:-2]
          output = '.'.join(output)
          # print(output)
          output += '. The relationship between the first node and the last node is {}.'.format(relationship)
          # print(output)
          if len(input) < 550:
            writer_icl.writerow({'input_text': input, 'output_text': output})
            count+=1



In [ ]:
# df1 = pd.read_csv('context.csv')
df1 = pd.read_csv('relation.csv')

df1.sample(5)

,input_text,output_text
1627,Answer the following multiple-choice question ...,"node_15105 has relation_0 with node_11747, mea..."
356,Answer the following multiple-choice question ...,"node_7016 has relation_0 with node_7017, means..."
4801,Answer the following multiple-choice question ...,"node_24368 has relation_0 with node_4442, mean..."
1816,Answer the following multiple-choice question ...,"node_32602 has relation_0 with node_23860, mea..."
1985,Answer the following multiple-choice question ...,"node_12341 has relation_4 with node_27052, mea..."


In [ ]:
import pandas as pd
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

preds1 = []
expec1 = []
input1 = []

# instruction = 'Answer the following yes/no question by reasoning step-by-step. '
# # prompt = 'Is the first node connnected with the last node?'
# prompt = 'Is the last node connnected with the first node?'



model_path = '/common/home/tc822/Desktop/last/results/checkpoint-3000'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

test_df = pd.read_csv('/common/home/tc822/Desktop/last/relation.csv')
test_df = test_df.sample(5)

correct = 0

total = 0
count = 0
for index, row in test_df.iterrows():
    input_text = row['input_text']
    expected_output = row['output_text']
# input_text = "node_11324 has relation_1 with node_11323. node_11323 has relation_1 with node_26334. " + instruction + prompt
# expected_output = "node_11324 has relation_1 with node_11323, means node_11324 _derivationally_related_form node_11323. node_11323 has relation_1 with node_26334, means node_11323 _derivationally_related_form node_26334. So node 11324 _derivationally_related_form node 26334. The answer is yes."
    # print(input_text)
    # print(expected_output)

    # 对输入进行编码
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # 进行预测并解码输出
    generation_config = {
        'max_new_tokens': 1500
    }

    outputs = model.generate(input_ids, **generation_config)
    # outputs = model.generate(input_ids)
    # output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(outputs)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(count, output_text.split()[-1])
    # count+=1
    # print(output_text.strip()[-5:])
    print(output_text)
    if len(output_text) > 0:
        output_text = output_text.split()[-1]
    else:
        output_text = 'no.'
    print(output_text)
    # preds1.append(output_text)
    # expec1.append(expected_output.split()[-1])
    # input1.append(input_text.split())

    output_text_new = output_text.replace(".", "").replace("_", "")
    expected_output_new = expected_output.split()[-1].replace(".", "").replace("_", "")
    # print(output_text_new, expected_output_new)

    preds1.append(output_text_new)
    expec1.append(expected_output_new)
    input1.append(input_text.split())

    if expected_output_new == output_text_new:
        correct += 1
    total += 1

accuracy = correct / total


print(f"Accuracy: {accuracy}")

preds_test1 = np.array([1 if 'no' in label else 0 for label in preds1])
expec_test1 = np.array([1 if 'no' in label else 0 for label in expec1])
# preds_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in preds])
# expec_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in expec])
f1 = f1_score(expec_test1, preds_test1)

# print(preds[:50])
# print(expec[:50])

auc = roc_auc_score(expec_test1, preds_test1)

print(f"F1: {f1}")
print(f"AUC: {auc}")

# *Relation prediction with In-context learning*




In [ ]:
import csv
import pandas as pd


In [ ]:
# context = 'Given node_2075 has relation_4 with node_12648. The relation between the first node and last node is _member_meronym. '
# instruction = 'Answer the following multiple-choice question by choosing one of these options: '
# option = ''
instruction = 'Answer the following question step-by-step. '
count = 0

question = 'The relationship between the first node and the last node is ?'
fieldnames = ['input_text', 'output_text']
with open("relation2id.txt", "r") as file:
    relations = int(file.readline())
    for line in file:
        relation, relation_id = line.strip().split("\t")
    #     instruction += relation + ', '
    # instruction = instruction[:-2] + '. '

with open("test.csv") as test:
    with open("test_icl.csv", mode="w", newline='') as icl:
      writer_icl = csv.DictWriter(icl, fieldnames=fieldnames)
      writer_icl.writeheader()
      reader = csv.reader(test)


      for row in reader:
          # print(row)
          input = row[0]
          output = row[1]
          if input == 'input_text' or output == 'output_text':
            continue
          input = input.split('.')
          input = input[:-2]
          input = '.'.join(input)
          # print(input)
          input = instruction + input + '. ' + question
          # input = input + '. ' + instruction + question

          output = output.split('.')
          words = output[-3].split()
          relationship = words[3]
          # print(relationship)
          output = output[:-2]
          output = '.'.join(output)
          # print(output)
          output += '. The relationship between the first node and the last node is {}.'.format(relationship)
          # print(output)
          if len(input) < 550:
            writer_icl.writerow({'input_text': input, 'output_text': output})
            count+=1



In [ ]:
c = 0

with open("train.csv") as train:
  with open("context.csv", mode="w", newline='') as context:
      writer_context = csv.DictWriter(context, fieldnames=fieldnames)
      writer_context.writeheader()
      reader_train = csv.reader(train)
      # Iterate over each row in the CSV file
      for row in reader_train:
          input = row[0]
          output = row[1]
          if input == 'input_text' or output == 'output_text':
            continue

          input = input.split('.')
          input = input[:-2]
          input = '.'.join(input)

          output = output.split('.')
          words = output[-3].split()
          relationship = words[3]

          if c < count and len(input) < 50:
            cont = 'Context: {}. The relationship between the first node and last node is {}.'.format(input, relationship)
            writer_context.writerow({'input_text': cont, 'output_text': relationship})
            c += 1
            # print(c)



In [ ]:

# read csv files
df1 = pd.read_csv('context.csv')
df2 = pd.read_csv('test_icl.csv')

# combine dataframes by interleaving rows

combined_df = pd.concat([df1, df2]).sort_index(kind='merge')

# write to new csv file
combined_df.to_csv('icl_relation.csv', index=False)



In [ ]:
# df1 = pd.read_csv('context.csv')
df1 = pd.read_csv('icl_relation.csv')

df1.sample(5)

,input_text,output_text
49,Answer the following multiple-choice question ...,"node_7670 has relation_0 with node_16080, mean..."
2207,Answer the following multiple-choice question ...,"node_16100 has relation_2 with node_6195, mean..."
4925,Answer the following multiple-choice question ...,"node_32920 has relation_0 with node_3721, mean..."
7954,Context: node_24739 has relation_0 with node_2...,_hypernym
2564,Context: node_3517 has relation_5 with node_69...,_synset_domain_topic_of


In [ ]:
import pandas as pd
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

preds1 = []
expec1 = []
input1 = []

# instruction = 'Answer the following yes/no question by reasoning step-by-step. '
# # prompt = 'Is the first node connnected with the last node?'
# prompt = 'Is the last node connnected with the first node?'



model_path = 'WN18RR/CoT/checkpoint-1000'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

test_df = pd.read_csv('WN18RR/CoT/icl_relation.csv')

correct = 0

total = 0
count = 0
for index, row in test_df.iterrows():
    input_text = row['input_text']
    expected_output = row['output_text']
# input_text = "node_11324 has relation_1 with node_11323. node_11323 has relation_1 with node_26334. " + instruction + prompt
# expected_output = "node_11324 has relation_1 with node_11323, means node_11324 _derivationally_related_form node_11323. node_11323 has relation_1 with node_26334, means node_11323 _derivationally_related_form node_26334. So node 11324 _derivationally_related_form node 26334. The answer is yes."
    # print(input_text)
    # print(expected_output)

    # 对输入进行编码
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # 进行预测并解码输出
    generation_config = {
        'max_new_tokens': 1500
    }

    outputs = model.generate(input_ids, **generation_config)
    # outputs = model.generate(input_ids)
    # output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(outputs)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(count, output_text.split()[-1])
    # count+=1
    # print(output_text.strip()[-5:])
    # print(output_text)
    if len(output_text) > 0:
        output_text = output_text.split()[-1]
    else:
        output_text = 'no.'
    # print(output_text)
    # preds1.append(output_text)
    # expec1.append(expected_output.split()[-1])
    # input1.append(input_text.split())

    output_text_new = output_text.replace(".", "").replace("_", "")
    expected_output_new = expected_output.split()[-1].replace(".", "").replace("_", "")
    # print(output_text_new, expected_output_new)

    preds1.append(output_text_new)
    expec1.append(expected_output_new)
    input1.append(input_text.split())

    if expected_output_new == output_text_new:
        correct += 1
    total += 1

accuracy = correct / total


print(f"Accuracy: {accuracy}")

preds_test1 = np.array([1 if 'no' in label else 0 for label in preds1])
expec_test1 = np.array([1 if 'no' in label else 0 for label in expec1])
# preds_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in preds])
# expec_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in expec])
f1 = f1_score(expec_test1, preds_test1)

# print(preds[:50])
# print(expec[:50])

auc = roc_auc_score(expec_test1, preds_test1)

print(f"F1: {f1}")
print(f"AUC: {auc}")

# *KG-LLM(Standard)*

In [ ]:
input_file = open("train2id.txt", "r")
output_file = open("output.txt", "w")

# total number of lines
number = int(input_file.readline())

nodes = set()

graph = {}

for i in range(number):
    content = input_file.readline()
    node1, node2, relation = content.strip().split()

    nodes.add(node1)
    # nodes.add(node2)

    relation = int(relation)

    # Check if the first node already exists in the dictionary
    if node1 not in graph:
        # If not, create a new dictionary for the node
        graph[node1] = {}
    # Add the neighboring node and the relationship to the dictionary for node1
    graph[node1][node2] = relation


node_list = list(nodes)
node_list2 = list(nodes)

In [ ]:

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# relation2id = {}

# with open("relation2id.txt", "r") as file:
#     relations = int(file.readline())
#     for line in file:
#         relation, relation_id = line.strip().split("\t")
#         relation2id[int(relation_id)] = relation

unique_rows = set()

# change size if you want, but larger the size, smaller the data
size = 30
# how many positive and negative data set you want to train, the more dataset, the more time to train
total = 60000
fieldnames = ['input_text', 'output_text']

# instruction = 'Answer the following yes/no question by reasoning step-by-step. '

with open("train_data.csv", mode="w", newline='') as tra:
  with open("positive_data.csv", mode="w", newline='') as pos:
    with open("negative_data.csv",  mode="w", newline='') as neg:

        # Create a CSV writer object and write the headers to the file
        writer_pos = csv.DictWriter(pos, fieldnames=fieldnames)
        writer_pos.writeheader()

        writer_neg = csv.DictWriter(neg, fieldnames=fieldnames)
        writer_neg.writeheader()

        writer_tra = csv.DictWriter(tra, fieldnames=fieldnames)
        writer_tra.writeheader()


        def dfs(graph, size):
            pos_count = 0
            neg_count = 0
            times = 0
            term = True

            while times < total:
                visited = set()
                kg = []
                graph_size = random.randint(2, size)
                first_node = random.choice(node_list)
                visited.add(first_node)
                last_node = ""
                previous_node = first_node
                stack = [first_node]
                input_text = ""
                output_text = ""
                while len(visited) < graph_size:
                    if previous_node not in graph or set(graph[previous_node].keys()).issubset(visited):
                        node = random.choice(node_list)
                        while node in visited:
                            node = random.choice(node_list)
                        input_text += 'node_{} not connected with node_{}. '.format(previous_node, node)
                        # output_text += 'node_{} not connected with node_{} means there is no relationship between node_{} and node_{}. '.format(previous_node, node, previous_node, node)
                        visited.add(node)
                        previous_node = node
                    else:
                        node = random.choice(list(graph[previous_node].keys()))
                        while node in visited:
                            node = random.choice(list(graph[previous_node].keys()))
                        relation = graph[previous_node][node]
                        # r = relation2id[relation]
                        input_text += 'node_{} has relation_{} with node_{}. '.format(previous_node, relation, node)
                        # output_text += 'node_{} has relation_{} with node_{}, means node_{} {} node_{}. '.format(previous_node, relation, node, previous_node, r, node)
                        visited.add(node)
                        previous_node = node
                    if len(visited) == graph_size:
                        last_node = previous_node

                # input_text += 'Answer the following yes/no question by reasoning step-by-step. Is the first node connnected with the last node?'
                was = len(unique_rows)
                unique_rows.add(input_text)
                if len(unique_rows) > was:
                  # if pos_count < int(total/2):
                    if first_node in graph and last_node in graph[first_node] and term:
                      # if pos_count < int(total/2):
                        # final_relation = relation2id[graph[first_node][last_node]]
                        output_text += 'The answer is yes.'
                        prompt = 'Is node {} connnected with node {}?'.format(first_node, last_node)
                        writer_pos.writerow({'input_text': input_text + prompt, 'output_text': output_text})
                        writer_tra.writerow({'input_text': input_text + prompt, 'output_text': output_text})
                        pos_count += 1
                        term = False
                        times += 1
                    elif last_node in graph and first_node in graph[last_node] and term:
                      # if pos_count < int(total/2):
                        # final_relation = relation2id[graph[last_node][first_node]]
                        output_text += 'The answer is yes.'
                        prompt = 'Is node {} connected with node {}?'.format(last_node, first_node)
                        writer_pos.writerow({'input_text': input_text + prompt, 'output_text': output_text})
                        writer_tra.writerow({'input_text': input_text + prompt, 'output_text': output_text})
                        pos_count += 1
                        term = False
                        times += 1
                  # elif neg_count < int(total//2):
                    elif not term:
                      # if neg_count < int(total/2):
                        output_text += 'The answer is no.'
                        prompt = 'Is node {} connected with node {}?'.format(first_node, last_node)
                        writer_neg.writerow({'input_text': input_text + prompt, 'output_text': output_text})
                        writer_tra.writerow({'input_text': input_text + prompt, 'output_text': output_text})
                        neg_count += 1
                        term = True
                        times += 1
                else:
                  continue


            print(pos_count)
            print(neg_count)

        dfs(graph, size)

30000
30000


In [ ]:
df1 = pd.read_csv('train_data.csv')
df1.sample(5)

,input_text,output_text
59443,node_28451 has relation_0 with node_17195. nod...,The answer is no.
18815,node_5873 has relation_5 with node_172. node_1...,The answer is no.
24541,node_5221 has relation_1 with node_23016. node...,The answer is no.
32618,node_18517 has relation_6 with node_34861. Is ...,The answer is yes.
26033,node_10117 has relation_0 with node_184. node_...,The answer is no.


In [ ]:
import csv

# Open the CSV file for reading and writing
with open('positive_data.csv', 'r+') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)

    # Create a set to store unique rows
    unique_rows = set()

    # Iterate over each row in the CSV file
    for row in reader:
        # Convert the row to a tuple and add it to the set
        unique_rows.add(tuple(row))

    # Clear the contents of the file and write the header row
    csvfile.seek(0)
    csvfile.truncate()
    writer = csv.writer(csvfile)
    writer.writerow(['input_text', 'output_text'])

    num_unique_rows = len(unique_rows)
    total_rows = reader.line_num

    print("Total rows:", total_rows)
    print("Unique rows:", num_unique_rows)

    # Write the unique rows to the file
    for row in unique_rows:
        writer.writerow(row)

Total rows: 30001
Unique rows: 30001


In [ ]:
# Open the CSV file for reading and writing
with open('negative_data.csv', 'r+') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)

    # Create a set to store unique rows
    unique_rows = set()

    # Iterate over each row in the CSV file
    for row in reader:
        # Convert the row to a tuple and add it to the set
        unique_rows.add(tuple(row))

    # Clear the contents of the file and write the header row
    csvfile.seek(0)
    csvfile.truncate()
    writer = csv.writer(csvfile)
    writer.writerow(['input_text', 'output_text'])

    num_unique_rows = len(unique_rows)
    total_rows = reader.line_num

    print("Total rows:", total_rows)
    print("Unique rows:", num_unique_rows)

    # Write the unique rows to the file
    for row in unique_rows:
        writer.writerow(row)


Total rows: 30001
Unique rows: 30001


In [ ]:
# Load the CSV file into a pandas DataFrame
data = pd.read_csv('positive_data.csv')

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2)

# Save the test data to a new CSV file
test_data.to_csv('pos_test.csv', index=False)

# Delete the test data from the original CSV file
data.drop(test_data.index, inplace=True)
data.to_csv('positive_train.csv', index=False)

In [ ]:
# Load the CSV file into a pandas DataFrame
data = pd.read_csv('negative_data.csv')

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2)

# Save the test data to a new CSV file
test_data.to_csv('neg_test.csv', index=False)

# Delete the test data from the original CSV file
data.drop(test_data.index, inplace=True)
data.to_csv('negative_train.csv', index=False)

In [ ]:
# combind the training data
df1 = pd.read_csv('positive_train.csv')
df2 = pd.read_csv('negative_train.csv')
combined_df = pd.concat([df1, df2], axis=0)
combined_df.to_csv('train.csv', index=False)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd

# Load the FLAN-T5 model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base', model_max_length=512)

# Define the dataset class
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        # Get the input and output sequences
        input_sequence = self.data.iloc[idx]['input_text']
        output_sequence = self.data.iloc[idx]['output_text']

        # Add task-specific prefix to input sequence
        # input_sequence = 'is first node connect with last node: ' + input_sequence

        # Encode the input and output sequences using the T5 tokenizer
        input_encoding = tokenizer(input_sequence, padding='max_length', max_length=128, truncation=True, return_tensors='pt')
        output_encoding = tokenizer(output_sequence, padding='max_length', max_length=128, truncation=True, return_tensors='pt')

        # Get the input IDs, attention mask, and label IDs from the encodings
        input_ids = input_encoding['input_ids'].squeeze()
        attention_mask = input_encoding['attention_mask'].squeeze()
        label_ids = output_encoding['input_ids'].squeeze()

        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'label_ids': label_ids}

    def __len__(self):
        return len(self.data)

# Define the data collator function
def data_collator(batch):
    input_ids = torch.stack([example['input_ids'] for example in batch])
    attention_mask = torch.stack([example['attention_mask'] for example in batch])
    label_ids = torch.stack([example['label_ids'] for example in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': label_ids}

# Load the data
preprocess_data = pd.read_csv('train.csv')

# Split the data into train and validation sets
train_data = preprocess_data.sample(frac=0.8, random_state=1)
val_data = preprocess_data.drop(train_data.index)

# Create the datasets
train_dataset = MyDataset(train_data)
val_dataset = MyDataset(val_data)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=1000,
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()


Link prediction without ICL

In [ ]:
import pandas as pd
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

preds1 = []
expec1 = []
input1 = []

# instruction = 'Answer the following yes/no question by reasoning step-by-step. '
# # prompt = 'Is the first node connnected with the last node?'
# prompt = 'Is the last node connnected with the first node?'



model_path = '/content/results/checkpoint-3000'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

test_df = pd.read_csv('/content/test.csv')
test_df = test_df.sample(1000)

correct = 0

total = 0
count = 0
for index, row in test_df.iterrows():
    input_text = row['input_text']
    expected_output = row['output_text']
# input_text = "node_11324 has relation_1 with node_11323. node_11323 has relation_1 with node_26334. " + instruction + prompt
# expected_output = "node_11324 has relation_1 with node_11323, means node_11324 _derivationally_related_form node_11323. node_11323 has relation_1 with node_26334, means node_11323 _derivationally_related_form node_26334. So node 11324 _derivationally_related_form node 26334. The answer is yes."
    # print(input_text)
    # print(expected_output)

    # 对输入进行编码
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # 进行预测并解码输出
    generation_config = {
        'max_new_tokens': 1500
    }

    outputs = model.generate(input_ids, **generation_config)
    # outputs = model.generate(input_ids)
    # output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(outputs)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(count, output_text.split()[-1])
    # count+=1
    # print(output_text.strip()[-5:])
    # print(output_text)
    if len(output_text) > 0:
        output_text = output_text.split()[-1]
    else:
        output_text = 'no.'
    # print(output_text)
    # preds1.append(output_text)
    # expec1.append(expected_output.split()[-1])
    # input1.append(input_text.split())

    output_text_new = output_text.replace(".", "").replace("_", "")
    expected_output_new = expected_output.split()[-1].replace(".", "").replace("_", "")
    # print(output_text_new, expected_output_new)

    preds1.append(output_text_new)
    expec1.append(expected_output_new)
    input1.append(input_text.split())

    if expected_output_new == output_text_new:
        correct += 1
    total += 1

accuracy = correct / total


print(f"Accuracy: {accuracy}")

preds_test1 = np.array([1 if 'no' in label else 0 for label in preds1])
expec_test1 = np.array([1 if 'no' in label else 0 for label in expec1])
# preds_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in preds])
# expec_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in expec])
f1 = f1_score(expec_test1, preds_test1)

# print(preds[:50])
# print(expec[:50])

auc = roc_auc_score(expec_test1, preds_test1)

print(f"F1: {f1}")
print(f"AUC: {auc}")

Link prediction with ICL

In [ ]:
import pandas as pd
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

preds1 = []
expec1 = []
input1 = []

# instruction = 'Answer the following yes/no question by reasoning step-by-step. '
# # prompt = 'Is the first node connnected with the last node?'
# prompt = 'Is the last node connnected with the first node?'



model_path = '/common/home/tc822/Desktop/last/results/checkpoint-3000'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

test_df = pd.read_csv('/common/home/tc822/Desktop/last/icl_link.csv')

correct = 0

total = 0
count = 0
for index, row in test_df.iterrows():
    input_text = row['input_text']
    expected_output = row['output_text']
# input_text = "node_11324 has relation_1 with node_11323. node_11323 has relation_1 with node_26334. " + instruction + prompt
# expected_output = "node_11324 has relation_1 with node_11323, means node_11324 _derivationally_related_form node_11323. node_11323 has relation_1 with node_26334, means node_11323 _derivationally_related_form node_26334. So node 11324 _derivationally_related_form node 26334. The answer is yes."
    # print(input_text)
    # print(expected_output)

    # 对输入进行编码
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # 进行预测并解码输出
    generation_config = {
        'max_new_tokens': 1500
    }

    outputs = model.generate(input_ids, **generation_config)
    # outputs = model.generate(input_ids)
    # output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(outputs)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(count, output_text.split()[-1])
    # count+=1
    # print(output_text.strip()[-5:])
    # print(output_text)
    if len(output_text) > 0:
        output_text = output_text.split()[-1]
    else:
        output_text = 'no.'
    # print(output_text)
    # preds1.append(output_text)
    # expec1.append(expected_output.split()[-1])
    # input1.append(input_text.split())

    output_text_new = output_text.replace(".", "").replace("_", "")
    expected_output_new = expected_output.split()[-1].replace(".", "").replace("_", "")
    # print(output_text_new, expected_output_new)

    preds1.append(output_text_new)
    expec1.append(expected_output_new)
    input1.append(input_text.split())

    if expected_output_new == output_text_new:
        correct += 1
    total += 1

accuracy = correct / total


print(f"Accuracy: {accuracy}")

preds_test1 = np.array([1 if 'no' in label else 0 for label in preds1])
expec_test1 = np.array([1 if 'no' in label else 0 for label in expec1])
# preds_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in preds])
# expec_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in expec])
f1 = f1_score(expec_test1, preds_test1)

# print(preds[:50])
# print(expec[:50])

auc = roc_auc_score(expec_test1, preds_test1)

print(f"F1: {f1}")
print(f"AUC: {auc}")

Relation prediction without ICL

In [ ]:
import pandas as pd
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

preds1 = []
expec1 = []
input1 = []

# instruction = 'Answer the following yes/no question by reasoning step-by-step. '
# # prompt = 'Is the first node connnected with the last node?'
# prompt = 'Is the last node connnected with the first node?'



model_path = '/common/home/tc822/Desktop/last/results/checkpoint-3000'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

test_df = pd.read_csv('/common/home/tc822/Desktop/last/relation.csv')
test_df = test_df.sample(5)

correct = 0

total = 0
count = 0
for index, row in test_df.iterrows():
    input_text = row['input_text']
    expected_output = row['output_text']
# input_text = "node_11324 has relation_1 with node_11323. node_11323 has relation_1 with node_26334. " + instruction + prompt
# expected_output = "node_11324 has relation_1 with node_11323, means node_11324 _derivationally_related_form node_11323. node_11323 has relation_1 with node_26334, means node_11323 _derivationally_related_form node_26334. So node 11324 _derivationally_related_form node 26334. The answer is yes."
    # print(input_text)
    # print(expected_output)

    # 对输入进行编码
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # 进行预测并解码输出
    generation_config = {
        'max_new_tokens': 1500
    }

    outputs = model.generate(input_ids, **generation_config)
    # outputs = model.generate(input_ids)
    # output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(outputs)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(count, output_text.split()[-1])
    # count+=1
    # print(output_text.strip()[-5:])
    print(output_text)
    if len(output_text) > 0:
        output_text = output_text.split()[-1]
    else:
        output_text = 'no.'
    print(output_text)
    # preds1.append(output_text)
    # expec1.append(expected_output.split()[-1])
    # input1.append(input_text.split())

    output_text_new = output_text.replace(".", "").replace("_", "")
    expected_output_new = expected_output.split()[-1].replace(".", "").replace("_", "")
    # print(output_text_new, expected_output_new)

    preds1.append(output_text_new)
    expec1.append(expected_output_new)
    input1.append(input_text.split())

    if expected_output_new == output_text_new:
        correct += 1
    total += 1

accuracy = correct / total


print(f"Accuracy: {accuracy}")

preds_test1 = np.array([1 if 'no' in label else 0 for label in preds1])
expec_test1 = np.array([1 if 'no' in label else 0 for label in expec1])
# preds_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in preds])
# expec_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in expec])
f1 = f1_score(expec_test1, preds_test1)

# print(preds[:50])
# print(expec[:50])

auc = roc_auc_score(expec_test1, preds_test1)

print(f"F1: {f1}")
print(f"AUC: {auc}")

Relation prediction with ICL

In [ ]:
import pandas as pd
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

preds1 = []
expec1 = []
input1 = []

# instruction = 'Answer the following yes/no question by reasoning step-by-step. '
# # prompt = 'Is the first node connnected with the last node?'
# prompt = 'Is the last node connnected with the first node?'



model_path = 'WN18RR/CoT/checkpoint-1000'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

test_df = pd.read_csv('WN18RR/CoT/icl_relation.csv')

correct = 0

total = 0
count = 0
for index, row in test_df.iterrows():
    input_text = row['input_text']
    expected_output = row['output_text']
# input_text = "node_11324 has relation_1 with node_11323. node_11323 has relation_1 with node_26334. " + instruction + prompt
# expected_output = "node_11324 has relation_1 with node_11323, means node_11324 _derivationally_related_form node_11323. node_11323 has relation_1 with node_26334, means node_11323 _derivationally_related_form node_26334. So node 11324 _derivationally_related_form node 26334. The answer is yes."
    # print(input_text)
    # print(expected_output)

    # 对输入进行编码
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # 进行预测并解码输出
    generation_config = {
        'max_new_tokens': 1500
    }

    outputs = model.generate(input_ids, **generation_config)
    # outputs = model.generate(input_ids)
    # output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(outputs)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(count, output_text.split()[-1])
    # count+=1
    # print(output_text.strip()[-5:])
    # print(output_text)
    if len(output_text) > 0:
        output_text = output_text.split()[-1]
    else:
        output_text = 'no.'
    # print(output_text)
    # preds1.append(output_text)
    # expec1.append(expected_output.split()[-1])
    # input1.append(input_text.split())

    output_text_new = output_text.replace(".", "").replace("_", "")
    expected_output_new = expected_output.split()[-1].replace(".", "").replace("_", "")
    # print(output_text_new, expected_output_new)

    preds1.append(output_text_new)
    expec1.append(expected_output_new)
    input1.append(input_text.split())

    if expected_output_new == output_text_new:
        correct += 1
    total += 1

accuracy = correct / total


print(f"Accuracy: {accuracy}")

preds_test1 = np.array([1 if 'no' in label else 0 for label in preds1])
expec_test1 = np.array([1 if 'no' in label else 0 for label in expec1])
# preds_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in preds])
# expec_test = np.array([0 if 'no' in label else 1 if 'yes' in label else 2 for label in expec])
f1 = f1_score(expec_test1, preds_test1)

# print(preds[:50])
# print(expec[:50])

auc = roc_auc_score(expec_test1, preds_test1)

print(f"F1: {f1}")
print(f"AUC: {auc}")